In [35]:
import dropbox
import glob
import pandas as pd
from bs4 import BeautifulSoup
import html5lib
import os
import re

In [17]:
file_path = os.getcwd()

In [30]:
def get_files(filepath):
    """
    The Function takes in current directory path as a parameter and process
    all files ending with extension .txtEach file in the directory represent 
    chick fil a locations for a given statethe function return a list of all the files processed 
    """
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.txt'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

In [33]:
def data_extraction(file):
    """
    the function takes in a file as a Parameter, then read
    and extract data from the HTML tags using BeautifulSoup Library.
    The data from one file is then Cleaned and added to a python file and returned 
    """
    all_data = []
    with open(file, "r") as f:
        content = f.read()
        soup = BeautifulSoup(content, 'lxml')
    soup.prettify()
    rows = soup.find_all("p")
    for row in rows:
        record = []
        for word in row.text.strip().split("\n"):
            if not word.isspace():
                record.append(re.sub("\s{2,}"," ",word.strip()))
        all_data.append(record)
    return all_data

In [37]:
def merge_all(path):
    """
    This function takes advantage of the above two functions to 
    process data from multiple files and merge them together into one repository
    """
    merged_files = []
    files = get_files(path)
    for f in files:
        locations = data_extraction(f)
        for location in locations[1:]:
            merged_files.append(location)
    return merged_files

In [38]:
data = merge_all(file_path)

In [40]:
max_size = max([len(x) for x in data])

In [41]:
# to ensure all files has the same number of columns
for record in data:
    if len(record) < max_size:
        record.extend(["None"]*(max_size-len(record)))

In [42]:
size = [len(r) for r in data]
max(size)

3

In [43]:
columns = ["address", "city", "State", "zip code", "telephone"]

In [45]:
new_data = []


In [46]:
# takes merged data and create a single dataFrame
for record in data[1:]:
    new_record = [None] * len(columns)
    if None not in record:
        for i in range(len(record)):
            if i == 1:
                if len(record[i].strip().split(",")) > 1:
                    new_record[i] = record[i].split(",")[0].split(" ")[-1] # city
                    new_record[i+1] = record[i].split(",")[1].strip().split(" ")[0] # state
                    new_record[i+2] = record[i].split(",")[1].strip().split(" ")[1] # zip
            elif i == 2:
                new_record[i+2] = record[i]
            else:
                new_record[i] = record[i]
            
    new_data.append(new_record)
        
    

In [48]:
df = pd.DataFrame(new_data, columns=columns)

In [50]:
# save data into csv file
df.to_csv("chick_fil_a_locations_data" + ".csv")

In [51]:
df.groupby(['city'])['address'].count().nlargest(10).reset_index(name="count")

,city,count
0,City,94
1,Atlanta,74
2,Beach,74
3,Houston,68
4,Charlotte,50
5,Park,44
6,Jacksonville,40
7,Columbia,36
8,Dallas,36
9,Springs,34
